In [2]:
import os
import glob 
import re
import pandas as pd
path = os.getcwd()

dat_files = glob.glob(os.path.join(path + "/dataset_cspro5", "[!~][!$]*.DAT"))
dcf_files = glob.glob(os.path.join(path + "/dataset_cspro5", "[!~][!$]*.DCF"))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
from pycspro import DictionaryParser
import pandas as pd
from pycspro import CaseParser
import json
from IPython.display import display

#check if string is actually string
def can_convert_to_int(string):
    try:
        int(string)
        return True
    except ValueError:
        return False
                            
                            
count = 0
count_col_name_occurence = {}

#To know if it is an ENT_QUEST table
in_table_counter = 0

for i in range(len(dat_files)):
    raw_dictionary = open(''+ dcf_files[i], 'r', encoding="utf8").read()
    dictionary_parser = DictionaryParser(raw_dictionary)
    parsed_dictionary = dictionary_parser.parse()
    
    raw_cases = open(''+ dat_files[i], 'r', encoding="utf8").read()
    cases = raw_cases.split('\n')
    case_parser = CaseParser(parsed_dictionary)
    parsed_cases = case_parser.parse(cases)

    
    dfs = {}
    for table_name, table in parsed_cases.items():
        print(table_name)
        dfs[table_name] = pd.DataFrame.from_dict(table)
        
        #Remove 'Aucun' columns
        try:
            dfs[table_name].drop('Aucun', axis=1, inplace=True)
            
        except Exception as e:
            pass
            
        
        #Delete subitems
        in_table_counter += 1
        if in_table_counter % 2 == 0:
            Items_ = parsed_dictionary['Dictionary']['Level']['Records'][0]['Items']
            for i2 in range(len(Items_)):
                col_name = Items_[i2]['Name']
                col_type = Items_[i2]['ItemType']

                if col_type == "SubItem":
                    try:
                        dfs[table_name].drop(col_name, axis=1, inplace=True)
                    except Exception as e:
                        pass
        
        
        # drop first row
        dfs[table_name] = dfs[table_name].iloc[1:, :]
        
        #Remove null columns
        try:
            nan_value = float("NaN")
            dfs[table_name].replace("", nan_value, inplace=True)
            dfs[table_name].dropna(how='all', axis=1, inplace=True)
                              
        except Exception as e:
            print("error null_ ", e)
            
                
        #Remove duplicate columns
        dfs[table_name] = dfs[table_name].T.drop_duplicates().T 
                
                
        #Treatement Proper       
        try:
            dfs[table_name].replace(dictionary_parser.get_value_labels(table_name), inplace=True)
            dfs[table_name].rename(columns = dictionary_parser.get_column_labels(table_name), inplace=True)
            
            #Get list of french columns '0'
            l1 = [x.split("|", 1)[0] if len(x.split("|", 1)) > 1 else x.split("|", 1)[0] for x in list(dfs[table_name].columns)]
            
            #Replace with list
            dfs[table_name].set_axis(l1, axis=1, inplace=True)
            
            #get values of columns in french
            dfs[table_name] = dfs[table_name].applymap(lambda x: x.split("|")[0] if type(x) != int and type(x) != float and len(x.split("|")) > 1 else x)
            
            
            #Unconverted table values
            #change table var
            data = dfs[table_name]
                

            for i_ in range(len(parsed_dictionary['Dictionary']['Level']['Records'][0]['Items'])):
                col_name = parsed_dictionary['Dictionary']['Level']['Records'][0]['Items'][i_]['Name']
                col_label = parsed_dictionary['Dictionary']['Level']['Records'][0]['Items'][i_]['Label']
                col_type = parsed_dictionary['Dictionary']['Level']['Records'][0]['Items'][i_]['ItemType']
                
                if col_type != "SubItem":
                    try:
                        l1 = parsed_dictionary['Dictionary']['Level']['Records'][0]['Items'][i_]['ValueSets'][0]['Value']
                        
                        
                        #dictionary with key as code and value as associate code value
                        dic = {}
                        #print(l1)
                        for i in l1:
                            try:
                                if len(i.split(";", 1)[0].split(" ", 1)[0]) > 1:
                                    dic[[int(i.split(";", 1)[0]) if i.split(";", 1)[0].isdigit() else i.split(";", 1)[0].split(" ", 1)[0].replace("'", "")][0]] = [i.split(";", 1)[1].split("|", 1)[0] if len(i.split(";", 1)[1].split("|", 1)) > 1 else i.split(";", 1)[1]][0]
                                else:
                                    dic[[int(i.split(";", 1)[0]) if i.split(";", 1)[0].isdigit() else i.split(";", 1)[0].replace("'", "")][0]] = [i.split(";", 1)[1].split("|", 1)[0] if len(i.split(";", 1)[1].split("|", 1)) > 1 else i.split(";", 1)[1]][0]

                            except Exception as e:
                                pass

                        #take label name in french
                        col_n = [col_label.split("|", 1)[0] if len(col_label.split("|", 1)) > 1 else col_label][0]
                        
                        #Values not found in dic but are found in data column
                        absent_vals_ = [a for a in data[col_n] if a not in dic.keys() and a not in dic.values()]
                        absent_vals = []
                        
                        #remove na values and duplicates
                        for val in absent_vals_:
                            if val not in absent_vals and pd.isnull(val) == False:
                                absent_vals.append(val)
                        
                        #for dictionary upgrade with corresponding values
                        dic2 = {}
                        if len(absent_vals) > 0:
                            if type(absent_vals[0]) == int and len(str(absent_vals[0])) == 1:
                                for val in absent_vals:
                                    dic2[val] = "Associate Not Found"
                                
                            #string that are ints
                            elif can_convert_to_int((absent_vals[0])):
                                #print("waiting for todo act 2")
                                #for val in absent_vals:
                                #    print(val)
                                #    print(dic)
                                #    print("\n\n")
                                pass
                                
                            #strings that are strings
                            elif can_convert_to_int((absent_vals[0])) == False:
                                for val in absent_vals:
                                    letter_val = " "
                                    letter_val2 = "Not Found "
                                    for letter in val:
                                        if letter != ' ':
                                            if letter not in dic:
                                                letter_val = letter_val + "" + str(letter_val2) + " & "
                                            else:
                                                letter_val = letter_val + "" + str(dic[letter]) + " & "
                                    dic2[val] = letter_val
                        
                                dic = {**dic, **dic2}
                                
                                
                        data = data.replace({col_n: dic})

                    except Exception as e:
                        pass
                  
                #Unconverted columns found in Quest tables data like regions, arrondissements...
                #try
                raw_dictionary_ = open('./dataset_cspro5/ENT_SEC1_ENT121_REC.DCF', 'r', encoding="utf8").read()
                dictionary_parser_ = DictionaryParser(raw_dictionary_)
                parsed_dictionary_ = dictionary_parser_.parse()

                for ii in range(len(parsed_dictionary_['Dictionary']['Level']['IdItems'])):
                    col_name_ = parsed_dictionary_['Dictionary']['Level']['IdItems'][ii]['Name']
                    col_label_ = parsed_dictionary_['Dictionary']['Level']['IdItems'][ii]['Label']
                    try:
                        l1 = parsed_dictionary_['Dictionary']['Level']['IdItems'][ii]['ValueSets'][0]['Value']
                        dic = {}
                        for j in l1:
                            dic[[int(j.split(";", 1)[0]) if j.split(";", 1)[0].isdigit() else j.split(";", 1)[0]][0]] = [j.split(";", 1)[1].split("|", 1)[0] if len(j.split(";", 1)[1].split("|", 1)) > 1 else j.split(";", 1)[1]][0]

                            
                        if [col_label_.split("|", 1)[0] if len(col_label_.split("|", 1)) > 1 else col_label_][0] == "Nom de la région":
                            data.replace({'Nom de la région': dic}, inplace=True)
                            
                        if [col_label_.split("|", 1)[0] if len(col_label_.split("|", 1)) > 1 else col_label_][0] == "Nom de l'arrondissement":
                            data.replace({"Nom de l'arrondissement": dic}, inplace=True)
                            
                        if [col_label_.split("|", 1)[0] if len(col_label_.split("|", 1)) > 1 else col_label_][0] == "Région":
                            data.replace({'Région': dic}, inplace=True)
                            
                        if [col_label_.split("|", 1)[0] if len(col_label_.split("|", 1)) > 1 else col_label_][0] == "Departement":
                            data.replace({"Département": dic}, inplace=True)
                            
                        if [col_label_.split("|", 1)[0] if len(col_label_.split("|", 1)) > 1 else col_label_][0] == "Arrondissement":
                            data.replace({"Arrondissement": dic}, inplace=True)
                            

                    except Exception as e:
                        pass

            if "REC" in table_name:
                data.to_csv("./FinalData5/"+ "Record_" + str(count) + ".csv", index=False, encoding="UTF-8-SIG")
            else:
                data.to_csv("./FinalData5/"+ table_name + "_" + str(count) + ".csv", index=False, encoding="UTF-8-SIG")

        except Exception as ee:
            count += 1
            dfs[table_name].replace(dictionary_parser.get_value_labels(table_name), inplace=True)
            dfs[table_name] = dfs[table_name].applymap(lambda x: x.split("|")[0] if type(x) != int and type(x) != float and len(x.split("|")) > 1 else x)
            
            
            #Unconverted tables(columns & values)
            raw_dictionary = open(''+ dcf_files[i], 'r', encoding="utf8").read()
            dictionary_parser = DictionaryParser(raw_dictionary)
            parsed_dictionary = dictionary_parser.parse()

            #error file
            #change er table var
            data = dfs[table_name]
        
                    
            for i in range(len(parsed_dictionary['Dictionary']['Level']['IdItems'])):
                col_name = parsed_dictionary['Dictionary']['Level']['IdItems'][i]['Name']
                col_label = parsed_dictionary['Dictionary']['Level']['IdItems'][i]['Label']
                
                try:
                    l1 = parsed_dictionary['Dictionary']['Level']['IdItems'][i]['ValueSets'][0]['Value']
                    dic = {}
                    for i in l1:
                        dic[[int(i.split(";", 1)[0]) if i.split(";", 1)[0].isdigit() else i.split(";", 1)[0]][0]] = [i.split(";", 1)[1].split("|", 1)[0] if len(i.split(";", 1)[1].split("|", 1)) > 1 else i.split(";", 1)[1]][0]

                    data = data.replace({col_name: dic})

                except Exception as e:
                    pass
                    
                finally:
                    #convert column names
                    data.rename(columns = { col_name: [col_label.split("|", 1)[0] if len(col_label.split("|", 1)) > 1 else col_label][0] }, inplace = True)

            if "REC" in table_name:
                data.to_csv("./FinalData5/"+ "Record_" + str(count) + "LM.csv", index=False, encoding="UTF-8-SIG")
            else:
                data.to_csv("./FinalData5/"+ table_name + str(count) + "LM.csv", index=False, encoding="UTF-8-SIG")

ENT_QUEST
ENT_AUT_INFOS_REC
ENT_QUEST
ENT_SEC0_REC
ENT_QUEST
ENT_SEC1_ENT121_REC
ENT_QUEST
ENT_SEC1_REC
ENT_QUEST
ENT_SEC2_ENT201_REC
ENT_QUEST
ENT_SEC2_ENT202_REC
ENT_QUEST
ENT_SEC2_REC
ENT_QUEST
ENT_SEC3A_REC
ENT_QUEST
ENT_SEC3B_ENT3B02_REC
ENT_QUEST
ENT_SEC3_REC
ENT_QUEST
ENT_SEC4A_REC
ENT_QUEST
ENT_SEC4B_REC
ENT_QUEST
ENT_SEC4C_REC
ENT_QUEST
ENT_SEC4EA_REC
ENT_QUEST
ENT_SEC4EB_REC
ENT_QUEST
ENT_SEC4_REC
ENT_QUEST
ENT_SEC5B_REC
ENT_QUEST
ENT_SEC5C_REC
ENT_QUEST
ENT_SEC5_REC
ENT_QUEST
ENT_SEC6_REC


In [3]:
ent_files = glob.glob(os.path.join(path + "/FinalData5", "[!~][!$]*.csv"))
n1 = 0
n2 = 0
for i in range(1, (len(ent_files)//2)+1):
    file1 = pd.read_csv("./FinalData5/ENT_QUEST"+str(i)+".csv", encoding="UTF-8")
    file2 = pd.read_csv("./FinalData5/Record_"+str(i)+".csv", encoding="UTF-8")
    
    file3 = pd.merge(file1, file2, left_index=True, right_index=True, suffixes=('', '_y'), how="outer")
    file3.drop(file3.filter(regex='_y$').columns, axis=1, inplace=True)
    file3.columns = file3.columns.str.replace(r"é", "e")
    file3.columns = file3.columns.str.replace(r"è", "e")
    file3.columns = file3.columns.str.replace(r"à", "a")
    file3.columns = file3.columns.str.replace(r"'", "")
    file3.columns = file3.columns.str.replace(r"â", "a")
    file3.columns = file3.columns.str.replace(r"ô", "O")
    
    file3 = file3.replace("é", "e", regex=True)
    file3 = file3.replace("è", "e", regex=True)
    file3 = file3.replace("à", "a", regex=True)
    file3 = file3.replace("'", "", regex=True)
    file3 = file3.replace("â", "a", regex=True)
    file3 = file3.replace("ô", "O", regex=True)
    file3 = file3.replace("/", " ou ", regex=True)
    #file3.replace(r"/", " ou ", inplace=True)
    print(file3.isnull().sum())
    n2 = n2 + file3.shape[1] - 4
    file3.to_csv("./FinalDataX/"+ "Record_" + str(i) + ".csv", index=False, encoding="UTF-8")

print(n2 + 4)

CASE_ID                                                  0
Region                                                   0
Departement                                              0
Arrondissement                                           0
Numero de l’entreprise dans larrondissement              0
Version                                                  0
du                                                       0
Etat de la collecte                                      0
Libelle de letat de la collecte                          0
Code du superviseur                                      0
Code de lagent                                           0
Mode dadministration                                    66
Date de depOt/ouverture du questionnaire (JJMMAAAA)     66
Date ed retrait/clOture du questionnaire (JJMMAAAA)    258
Date de debut de saisie (JJMMAAAA)                      65
Heure de debut de saisie (HHMMSS)                       65
Date de fin de saisie (JJMMAAAA)                       2

FileNotFoundError: [Errno 2] No such file or directory: './FinalData5/ENT_QUEST21.csv'